In [1]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore

cred = credentials.Certificate(
    "../../Documents/GoogleCerts/astroworld-trevor-firebase-adminsdk-l05ps-820638c3ff.json"
)
firebase_admin.initialize_app(cred)
db = firestore.client()


def name_star(row):
    name = row["proper"]
    hd = row["hd"]
    if pd.isna(name):
        if pd.isna(hd):
            return None
        else:
            return f"HD{int(hd)}"
    else:
        return name


def convert_dist(parsecs):
    au = parsecs * 206264.80624
    return au


table = pd.read_csv("./hyg_v42.csv")
table = table.sort_values("mag")
table = table[table["proper"] != "Sol"]
table = table.head(500)
table["proper"] = table.apply(name_star, axis=1)
table["dist"] = table["dist"].apply(convert_dist)
stars_table = table[
    ["id", "proper", "con", "dist", "dec", "ra", "mag", "lum", "spect"]
].rename(
    columns={
        "id": "id",
        "proper": "name",
        "con": "constellation",
        "dist": "distance",
        "dec": "declination",
        "ra": "rightAscension",
        "mag": "magnitude",
        "lum": "luminosity",
        "spect": "spectralType",
    }
)
stars_dict = stars_table.to_dict(orient="records")

objects_ref = db.collection("objects")
stars_doc = objects_ref.document("stars")
stars_doc.delete()

objects_ref.document("stars").set({"data": stars_dict})
print(f"{len(stars_dict)} Stars Uploaded Successfully.")

500 Stars Uploaded Successfully.
